In [1]:
using JLD
include("gyncycle.jl");

In [15]:
hillind = [4, 6, 10, 18, 20, 22, 26, 33, 36, 39, 43, 47, 49, 52, 55, 59, 65, 95, 98, 101, 103]
SAMPLEPARMS = deleteat!(collect(1:103), hillind)

SIGMA_RHO = 0.2
SIGMA_Y0 = 10
SIGMA_PARMS = 10;
persons = 1:3

s=0.01

parms, y0 = loadparms()
samplingnodes = [:sparms, :y0]
data = loadpfizer()
proposalvariance = diagm(1./vcat(parms[SAMPLEPARMS], y0));
proposalvariance = eye(length(SAMPLEPARMS)+length(y0)) * log(1+(s*s));

In [14]:
log(1)

0.0

In [ ]:
mc = []
for person in persons
    m, inp, ini = gyncmodel(data[person], parms, y0)
    setsamplers!(m, [AMM(samplingnodes, proposalvariance, adapt=:none)])
    
    push!(mc, mcmc(m, inp, ini, 100, burnin=0, verbose = false, thin = 1))
end

while size(mc[1],1) < 1_000_000
    for person in persons
        @time mc[person] = mcmc(mc[person], 100, verbose=false)        
    end
    save("lognorm-notimeshift.jld", "mc" , [mc[person].value for person in persons])
end